In [1]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py


In [2]:
# chemin des images
folder_path = 'C:/Users/Nans/OneDrive/Documents/Rakuten Project/images/images/image_train'


In [3]:
def img_cropping(img):

    largeur_fixe = 100
    hauteur_fixe = 100
    
    # Convertir l'image en niveaux de gris
    image_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Inverser les niveaux de gris
    image_inversee = cv2.bitwise_not(image_gris)

    # Trouver les contours dans l'image
    contours, hierarchy = cv2.findContours(image_inversee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        
        # Trouver le contour le plus externe
        contour_externe = max(contours, key=cv2.contourArea)

        # Trouver les coordonnées du rectangle englobant le contour externe
        x, y, w, h = cv2.boundingRect(contour_externe)

        # Rogner l'image en utilisant les coordonnées du rectangle englobant
        cropped_image = img[y:y+h, x:x+w]

        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

        # Redimensionner l'image pour qu'elle ait la même taille que les autres
        cropped_image_resized = cv2.resize(cropped_image, (largeur_fixe, hauteur_fixe))

        return np.array(cropped_image_resized)

    else:
        # Si aucun contour n'est trouvé, retourner None
        return None



In [5]:

# Obtenez la liste des fichiers dans le dossier
img_in_folder = os.listdir(folder_path)

# Parcourez chaque fichier dans le dossier
batch_size = 10000
num_batches = len(img_in_folder) // batch_size + 1
#num_batches = 2

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(img_in_folder))
    
    batch_files = img_in_folder[start_idx:end_idx]

    # Initialisez des listes pour stocker les identifiants des images et des produits
    image_ids = []
    product_ids = []
    image_vectors = []  
    
    # Traitement du lot d'images
    for img in batch_files:

        # Utilisez une expression régulière pour extraire les identifiants d'image et de produit
        match = re.match(r'image_(\d+)_product_(\d+)\.jpg', img)
        if match:
            image_id = match.group(1)
            product_id = match.group(2)
        
            # Lire l'image avec OpenCV
            img_path = os.path.join(folder_path, img)
            image = cv2.imread(img_path)

            # Fonction d'image cropping
            cropped_img = img_cropping(image)
        
            # Assurez-vous que l'image existe
            if cropped_img is not None:           
                # Ajoutez les identifiants et le vecteur d'image à leurs listes respectives
                image_ids.append(int(image_id))
                product_ids.append(int(product_id))
                image_vectors.append(cropped_img)
            
    # Convertir les listes en tableaux NumPy
    image_vectors = np.array(image_vectors)
    image_ids = np.array(image_ids)
    product_ids = np.array(product_ids)


    # Créer un fichier HDF5 et y enregistrer un tableau NumPy
    with h5py.File('data_' + str(batch_idx) + 'comp_100.h5', 'w') as hf:
        hf.create_dataset('X_images_train', data=image_vectors, compression='gzip')
        hf.create_dataset('image_ids', data=image_ids, compression='gzip')
        hf.create_dataset('product_ids', data=product_ids, compression='gzip')


In [6]:
class GrayScaleConverter(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
        

In [7]:
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(14,14),
                 cells_per_block=(2,2), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])
            

In [8]:
class CannyEdgeDetector(BaseEstimator, TransformerMixin):
    
    def __init__(self, gaussian_kernel_size=(3, 3), gaussian_sigma=0, canny_low_threshold=100, canny_high_threshold=200):
        self.gaussian_kernel_size = gaussian_kernel_size
        self.gaussian_sigma = gaussian_sigma
        self.canny_low_threshold = canny_low_threshold
        self.canny_high_threshold = canny_high_threshold
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        edged_images = []
        for image in X:
            # Appliquer le filtre gaussien
            blurred_image = cv2.GaussianBlur(np.uint8(image), self.gaussian_kernel_size, self.gaussian_sigma)
            
            # Appliquer le filtre Canny
            edges = cv2.Canny(blurred_image, self.canny_low_threshold, self.canny_high_threshold)
            
            # Ajouter les bords détectés à l'image d'origine
            edged_image = cv2.bitwise_and(image, image, mask=edges)
            
            edged_images.append(edged_image)
            
        return np.array(edged_images)


In [9]:
class LaplacianFilter(BaseEstimator, TransformerMixin):
    def __init__(self, ddepth=cv2.CV_64F):
        self.ddepth = ddepth
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        laplacian_images = []
        for image in X:
            # Appliquer le filtre Laplacien
            laplacian_image = cv2.Laplacian(np.uint8(image), self.ddepth)
            
            # Convertir l'image en 3 canaux si nécessaire
            if len(image.shape) == 3:
                laplacian_image = cv2.cvtColor(laplacian_image, cv2.COLOR_GRAY2BGR)
            
            laplacian_images.append(laplacian_image)
            
        return np.array(laplacian_images)


In [10]:
class HSVHistogram(BaseEstimator, TransformerMixin):
    def __init__(self, bins=256):
        self.bins = bins
    
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        histograms = []
        for image in X:
            
            # Assurez-vous que l'image est en format flottant
            image = image.astype(np.float32)

            # Normaliser l'image pour obtenir des valeurs entre 0 et 1
            image /= 255.0

            # Diviser l'image en canaux de couleur
            channels = cv2.split(image)

            # Initialiser un histogramme vide pour chaque canal de couleur
            hist = []

            # Calculer l'histogramme pour chaque canal de couleur
            for channel in channels:
                hist_channel, _ = np.histogram(channel, bins=256, range=(0, 1))
                hist.append(hist_channel)

            # Concaténer les histogrammes de chaque canal pour obtenir l'histogramme global
            hist = np.concatenate(hist)
            histograms.append(hist)
            
        return np.array(histograms)


In [11]:
# Charger un tableau NumPy à partir du fichier HDF5
with h5py.File('data_0_comp_100.h5', 'r') as hf:
    X_images_train = hf['X_images_train'][:]
    

In [12]:
processor = GrayScaleConverter()
X_train_gray = processor.transform(X_images_train)

processor = HogTransformer()
X_train_hog = processor.transform(X_train_gray)

processor = CannyEdgeDetector()
X_train_canny = processor.transform(X_train_gray)

processor = LaplacianFilter()
X_train_laplacian = processor.transform(X_train_gray)

processor = HSVHistogram()
X_train_hsv = processor.transform(X_images_train)


In [ ]:
# Ouvrir le fichier HDF5 existant en mode append
with h5py.File('data_0_comp_100.h5', 'r+') as hf:
    # Créer un nouvel ensemble de données dans le fichier HDF5
    hf.create_dataset('X_train_canny', data=X_train_canny, compression='gzip')
    hf.create_dataset('X_train_laplacian', data=X_train_laplacian, compression='gzip')
    hf.create_dataset('X_train_hog', data=X_train_hog, compression='gzip')
    hf.create_dataset('X_train_hsv', data=X_train_hsv, compression='gzip')
    hf.create_dataset('X_train_gray', data=X_train_gray, compression='gzip')
    

In [ ]:
del X_images_train
del X_train_gray
del X_train_canny
del X_train_laplacian
del X_train_hog
del X_train_hsv